# On click callback

Once all cells are executed, click anywhere to get the closest star name. Be patient and the associated photometric points appear after a few seconds.

In [1]:
from ipyaladin import Aladin
import requests
from ipywidgets import Layout, Box, widgets

aladin = Aladin(layout=Layout(width="70%"), target="M 36", fov=0.3)
info = widgets.HTML()


box_layout = Layout(
    display="flex", flex_flow="row", align_items="stretch", width="100%"
)
box = Box(children=[aladin, info], layout=box_layout)
box

Box(children=(<ipyaladin.widget.Aladin object at 0x7f498ba55e80>, HTML(value='')), layout=Layout(align_items='…

In [2]:
def process_result(data: dict) -> None:
    """Process the result of a click event on the Aladin widget.

    Parameters
    ----------
    data : dict
        The data returned by the click event.
    """
    info.value = ""
    ra = data["ra"]
    dec = data["dec"]
    query = """SELECT TOP 1 main_id, ra, dec, DISTANCE(POINT('ICRS', %f, %f),
               POINT('ICRS', ra, dec)) as d FROM basic
               WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', %f, %f, %f))=1
               ORDER BY d ASC""" % (
        ra,
        dec,
        ra,
        dec,
        aladin.fov.deg / 10,
    )

    r = requests.get(
        "http://simbad.u-strasbg.fr/simbad/sim-tap/sync",
        params={
            "query": query,
            "request": "doQuery",
            "lang": "adql",
            "format": "json",
            "phase": "run",
        },
    )
    obj_name = ""
    obj_coo = None
    obj_data = r.json()["data"]
    if len(obj_data) == 0:
        return

    obj_name = obj_data[0][0]
    obj_coo = [obj_data[0][1], obj_data[0][2]]

    sed_img = (
        '<img src="http://cdsportal.u-strasbg.fr/cgi-bin/PhotVizPreview/plot?ra=%f&dec=%f&radius_arcsec=5&w=200&h=150&point_size=4">'
        % (obj_coo[0], obj_coo[1])
    )
    info.value = "<h2>%s</h2><br><br>%s" % (obj_name, sed_img)


aladin.set_listener("click", process_result)